In [1]:
%reload_ext autotime
import geopandas as gpd
import pandas as pd
from glob import glob
from sklearn.linear_model import LinearRegression
from tqdm.auto import tqdm
from tqdm.contrib.concurrent import process_map
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, root_mean_squared_error
from coastsat import SDS_transects
from smoothn import smoothn

In [2]:
# Transects, origin is landward. Has beach_slope
transects = gpd.read_file("transects_extended.geojson")
transects.set_index("id", inplace=True)
transects

,site_id,orientation,along_dist,along_dist_norm,beach_slope,cil,ciu,trend,n_points,n_points_nonan,r2_score,mae,mse,rmse,intercept,ERODIBILITY,geometry
id,,,,,,,,,,,,,,,,,
aus0001-0000,aus0001,104.347648,0.000000,0.000000,0.085,0.0545,0.2000,-1.456305,654.0,342.0,0.131951,28.721214,1321.059410,36.346381,179.407277,None,"LINESTRING (153.26555 -24.7007, 153.26938 -24...."
aus0001-0001,aus0001,93.495734,98.408334,0.002935,0.050,0.0387,0.0640,-0.833581,654.0,473.0,0.052783,26.029482,1092.336862,33.050520,210.629160,None,"LINESTRING (153.26525 -24.7019, 153.2692 -24.7..."
aus0001-0002,aus0001,82.069341,198.408334,0.005918,0.050,0.0428,0.0647,-0.282209,654.0,491.0,0.008220,22.896464,838.997233,28.965449,200.166307,None,"LINESTRING (153.26539 -24.70316, 153.26931 -24..."
aus0001-0003,aus0001,81.192757,298.402523,0.008900,0.055,0.0480,0.0659,0.089340,654.0,502.0,0.000987,20.770475,691.179232,26.290288,185.295473,None,"LINESTRING (153.26555 -24.70408, 153.26945 -24..."
aus0001-0004,aus0001,81.065473,398.402523,0.011882,0.075,0.0614,0.0922,0.361437,654.0,508.0,0.017461,19.528839,637.676213,25.252252,169.027861,None,"LINESTRING (153.2657 -24.70497, 153.26961 -24...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sar2538-0004,sar2538,NaN,NaN,NaN,NaN,NaN,NaN,-0.085726,1026.0,942.0,0.022495,3.801042,33.841662,5.817359,245.090597,Low,"LINESTRING (9.33568 41.29334, 9.33294 41.29629)"
sar2539-0000,sar2539,NaN,NaN,NaN,NaN,NaN,NaN,0.284823,374.0,330.0,0.047681,6.193424,102.157484,10.107299,209.076772,Low,"LINESTRING (9.40022 41.3002, 9.40172 41.30362)"
sar2539-0001,sar2539,NaN,NaN,NaN,NaN,NaN,NaN,0.219223,374.0,352.0,0.040894,5.224866,71.772579,8.471870,189.484658,Low,"LINESTRING (9.39917 41.3019, 9.4035 41.30039)"


In [3]:
vos_files = pd.Series(
    sorted(glob("shoreline_data_run6/*/time_series_tidally_corrected.csv"))
)
vos_files = vos_files[~vos_files.str.contains("nzd")]
vos_files

0       shoreline_data_run6/aus0001/time_series_tidall...
1       shoreline_data_run6/aus0002/time_series_tidall...
2       shoreline_data_run6/aus0003/time_series_tidall...
3       shoreline_data_run6/aus0004/time_series_tidall...
4       shoreline_data_run6/aus0005/time_series_tidall...
                              ...                        
3064    shoreline_data_run6/usa_HI_0051/time_series_ti...
3065    shoreline_data_run6/usa_HI_0052/time_series_ti...
3066    shoreline_data_run6/usa_HI_0053/time_series_ti...
3067    shoreline_data_run6/usa_HI_0054/time_series_ti...
3068    shoreline_data_run6/usa_HI_0057/time_series_ti...
Length: 2509, dtype: object

In [4]:
my_nz_files = pd.Series(
    sorted(glob("data/*/transect_time_series_tidally_corrected.csv"))
)
my_nz_files

0      data/nzd0001/transect_time_series_tidally_corr...
1      data/nzd0002/transect_time_series_tidally_corr...
2      data/nzd0003/transect_time_series_tidally_corr...
3      data/nzd0004/transect_time_series_tidally_corr...
4      data/nzd0005/transect_time_series_tidally_corr...
                             ...                        
557    data/nzd0559/transect_time_series_tidally_corr...
558    data/nzd0560/transect_time_series_tidally_corr...
559    data/nzd0561/transect_time_series_tidally_corr...
560    data/nzd0562/transect_time_series_tidally_corr...
561    data/nzd0563/transect_time_series_tidally_corr...
Length: 562, dtype: object

In [5]:
sar_files = pd.Series(sorted(glob("data/sar*/transect_time_series.csv")))
sar_files

0       data/sar0001/transect_time_series.csv
1       data/sar0002/transect_time_series.csv
2       data/sar0003/transect_time_series.csv
3       data/sar0004/transect_time_series.csv
4       data/sar0005/transect_time_series.csv
                        ...                  
2534    data/sar2535/transect_time_series.csv
2535    data/sar2536/transect_time_series.csv
2536    data/sar2537/transect_time_series.csv
2537    data/sar2538/transect_time_series.csv
2538    data/sar2539/transect_time_series.csv
Length: 2539, dtype: object

In [6]:
files = pd.concat([vos_files, my_nz_files, sar_files])
files

0       shoreline_data_run6/aus0001/time_series_tidall...
1       shoreline_data_run6/aus0002/time_series_tidall...
2       shoreline_data_run6/aus0003/time_series_tidall...
3       shoreline_data_run6/aus0004/time_series_tidall...
4       shoreline_data_run6/aus0005/time_series_tidall...
                              ...                        
2534                data/sar2535/transect_time_series.csv
2535                data/sar2536/transect_time_series.csv
2536                data/sar2537/transect_time_series.csv
2537                data/sar2538/transect_time_series.csv
2538                data/sar2539/transect_time_series.csv
Length: 5610, dtype: object

In [7]:
"""f = sar_files.iloc[0]
despiked_filename = f.replace(".csv", "_despiked.csv")
df = pd.read_csv(despiked_filename)
df.dates = pd.to_datetime(df.dates)
df.set_index("dates", inplace=True)
import matplotlib.pyplot as plt
for transect_id in df.columns:
  sample = df[transect_id]
  ax = sample.plot(figsize=(10,10), style=".-", label="raw data")
  sample.rolling(10, min_periods=1).mean().plot(ax=ax, label="rolling mean")
  ax.legend()
  plt.show()"""

'f = sar_files.iloc[0]\ndespiked_filename = f.replace(".csv", "_despiked.csv")\ndf = pd.read_csv(despiked_filename)\ndf.dates = pd.to_datetime(df.dates)\ndf.set_index("dates", inplace=True)\nimport matplotlib.pyplot as plt\nfor transect_id in df.columns:\n  sample = df[transect_id]\n  ax = sample.plot(figsize=(10,10), style=".-", label="raw data")\n  sample.rolling(10, min_periods=1).mean().plot(ax=ax, label="rolling mean")\n  ax.legend()\n  plt.show()'

In [8]:
def despike(chainage, threshold=40):
    chainage = chainage.dropna()
    chainage, dates = SDS_transects.identify_outliers(
        chainage.tolist(), chainage.index.tolist(), threshold
    )
    return pd.Series(chainage, index=dates)


def get_trends(f):
    df = pd.read_csv(f)
    try:
        df.dates = pd.to_datetime(df.dates)
    except:
        print(f)
    if "sar" in f:
        despiked_filename = f.replace(".csv", "_despiked.csv")
        try:
            raise
            df = pd.read_csv(despiked_filename)
            df.dates = pd.to_datetime(df.dates)
        except:
            df.dates = pd.to_datetime(df.dates)
            df.set_index("dates", inplace=True)
            satname = df.satname
            df = df.drop(columns="satname").apply(despike, axis=0)
            df["satname"] = satname
            df.reset_index(names="dates", inplace=True)
            df.to_csv(f.replace(".csv", "_despiked.csv"), index=False)
    df.index = (df.dates - df.dates.min()).dt.days / 365.25
    df.drop(columns=["dates", "satname", "Unnamed: 0"], inplace=True, errors="ignore")
    trends = []
    for transect_id in df.columns:
        sub_df = df[transect_id].dropna()
        if not len(sub_df):
            continue
        x = sub_df.index.to_numpy().reshape(-1, 1)
        y = sub_df
        linear_model = LinearRegression().fit(x, y)
        pred = linear_model.predict(x)
        trends.append(
            {
                "transect_id": transect_id,
                "trend": linear_model.coef_[0],
                "intercept": linear_model.intercept_,
                "n_points": len(df[transect_id]),
                "n_points_nonan": len(sub_df),
                "r2_score": r2_score(y, pred),
                "mae": mean_absolute_error(y, pred),
                "mse": mean_squared_error(y, pred),
                "rmse": root_mean_squared_error(y, pred),
            }
        )
    return pd.DataFrame(trends)


trends = pd.concat(process_map(get_trends, files)).set_index("transect_id")
len(trends)

/tmp/ipykernel_2587794/144919997.py:56: TqdmWarning: Iterable length 5610 > 1000 but `chunksize` is not set. This may seriously degrade multiprocess performance. Set `chunksize=1` or more.
  trends = pd.concat(process_map(get_trends, files)).set_index("transect_id")


  0%|          | 0/5610 [00:00<?, ?it/s]

****************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************

/home/ubuntu/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


****************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************

141686

In [9]:
trends[trends.n_points_nonan > 10].sort_values("r2_score")

,trend,intercept,n_points,n_points_nonan,r2_score,mae,mse,rmse
transect_id,,,,,,,,
sar1247-0000,-4.976915e-07,213.492243,676,390,4.544143e-13,7.008953,79.269336,8.903333
aus0213-0000,-2.941492e-06,211.475107,543,487,8.622880e-12,7.797467,94.136738,9.702409
chi0406-0028,5.363548e-06,183.818769,456,396,1.916567e-11,6.755653,85.309469,9.236312
sar0065-0007,-3.687583e-06,250.663947,1351,1226,3.013378e-11,6.684365,66.839643,8.175552
mex0157-0034,-8.100513e-06,225.225194,496,413,3.072342e-11,10.834152,183.654195,13.551907
...,...,...,...,...,...,...,...,...
sar0811-0000,-7.032405e+00,362.639306,555,485,9.855151e-01,7.916487,96.965647,9.847114
sar0812-0006,-6.575311e+00,355.339454,497,406,9.858499e-01,7.330929,83.405010,9.132634
sar0811-0004,-7.012166e+00,390.632125,555,483,9.875653e-01,6.656741,69.992496,8.366152


In [10]:
trends[trends.index.str.startswith("sar")].describe()

,trend,intercept,n_points,n_points_nonan,r2_score,mae,mse,rmse
count,19753.000000,19753.000000,19753.000000,19753.000000,1.975200e+04,19753.000000,19753.000000,19753.000000
mean,0.017346,226.961934,958.027034,835.533792,6.486554e-02,9.902562,478.543039,13.350296
std,0.888066,95.636408,304.699574,260.981267,1.270734e-01,13.577256,2478.023185,17.329970
min,-21.444328,-142.041607,186.000000,1.000000,4.544143e-13,0.000000,0.000000,0.000000
25%,-0.110720,185.278389,683.000000,625.000000,3.300452e-03,5.067615,46.332703,6.806813
50%,-0.006963,203.587222,917.000000,735.000000,1.626634e-02,6.363024,69.997329,8.366441
75%,0.086853,226.977493,1280.000000,1111.000000,5.864840e-02,8.936355,135.507593,11.640773
max,13.317474,998.024507,1431.000000,1306.000000,9.919240e-01,317.242582,102265.506178,319.789784


In [11]:
trends.describe()

,trend,intercept,n_points,n_points_nonan,r2_score,mae,mse,rmse
count,141686.000000,141686.000000,141686.000000,141686.000000,1.416850e+05,141686.000000,141686.000000,141686.000000
mean,0.207585,237.022061,614.355342,492.880270,1.560859e-01,12.524839,372.356134,15.953587
std,1.907323,97.184282,297.143406,257.423456,2.119841e-01,8.687208,1117.233192,10.855415
min,-61.721935,-2765.025713,56.000000,1.000000,4.544143e-13,0.000000,0.000000,0.000000
25%,-0.228220,185.615804,405.000000,308.000000,1.144836e-02,7.947814,103.530590,10.174998
50%,0.078735,214.772076,551.000000,445.000000,5.920339e-02,10.584154,179.686445,13.404717
75%,0.525287,296.362341,745.000000,627.000000,2.129562e-01,14.271287,326.841363,18.078754
max,89.734302,2329.832271,1567.000000,1334.000000,9.919240e-01,317.242582,102265.506178,319.789784


In [12]:
(transects.trend - trends.trend).describe()

count    141708.000000
mean         -0.003215
std           0.135880
min         -14.617460
25%           0.000000
50%           0.000000
75%           0.000000
max           5.132658
Name: trend, dtype: float64

In [13]:
transects.update(trends.drop_duplicates())

In [14]:
transects

,site_id,orientation,along_dist,along_dist_norm,beach_slope,cil,ciu,trend,n_points,n_points_nonan,r2_score,mae,mse,rmse,intercept,ERODIBILITY,geometry
id,,,,,,,,,,,,,,,,,
aus0001-0000,aus0001,104.347648,0.000000,0.000000,0.085,0.0545,0.2000,-1.456305,654.0,342.0,0.131951,28.721214,1321.059410,36.346381,179.407277,None,"LINESTRING (153.26555 -24.7007, 153.26938 -24...."
aus0001-0001,aus0001,93.495734,98.408334,0.002935,0.050,0.0387,0.0640,-0.833581,654.0,473.0,0.052783,26.029482,1092.336862,33.050520,210.629160,None,"LINESTRING (153.26525 -24.7019, 153.2692 -24.7..."
aus0001-0002,aus0001,82.069341,198.408334,0.005918,0.050,0.0428,0.0647,-0.282209,654.0,491.0,0.008220,22.896464,838.997233,28.965449,200.166307,None,"LINESTRING (153.26539 -24.70316, 153.26931 -24..."
aus0001-0003,aus0001,81.192757,298.402523,0.008900,0.055,0.0480,0.0659,0.089340,654.0,502.0,0.000987,20.770475,691.179232,26.290288,185.295473,None,"LINESTRING (153.26555 -24.70408, 153.26945 -24..."
aus0001-0004,aus0001,81.065473,398.402523,0.011882,0.075,0.0614,0.0922,0.361437,654.0,508.0,0.017461,19.528839,637.676213,25.252252,169.027861,None,"LINESTRING (153.2657 -24.70497, 153.26961 -24...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sar2538-0004,sar2538,NaN,NaN,NaN,NaN,NaN,NaN,-0.075520,1217.0,1129.0,0.021249,4.034425,37.749282,6.144044,244.758910,Low,"LINESTRING (9.33568 41.29334, 9.33294 41.29629)"
sar2539-0000,sar2539,NaN,NaN,NaN,NaN,NaN,NaN,0.159357,388.0,337.0,0.014258,6.685558,118.654858,10.892881,212.777700,Low,"LINESTRING (9.40022 41.3002, 9.40172 41.30362)"
sar2539-0001,sar2539,NaN,NaN,NaN,NaN,NaN,NaN,0.160824,388.0,356.0,0.020390,5.452777,82.388602,9.076817,191.174425,Low,"LINESTRING (9.39917 41.3019, 9.4035 41.30039)"


In [15]:
trends.columns, transects.columns, trends.columns.isin(transects.columns)

(Index(['trend', 'intercept', 'n_points', 'n_points_nonan', 'r2_score', 'mae',
        'mse', 'rmse'],
       dtype='object'),
 Index(['site_id', 'orientation', 'along_dist', 'along_dist_norm',
        'beach_slope', 'cil', 'ciu', 'trend', 'n_points', 'n_points_nonan',
        'r2_score', 'mae', 'mse', 'rmse', 'intercept', 'ERODIBILITY',
        'geometry'],
       dtype='object'),
 array([ True,  True,  True,  True,  True,  True,  True,  True]))

In [16]:
transects = transects.join(trends.loc[:, ~trends.columns.isin(transects.columns)])
transects

,site_id,orientation,along_dist,along_dist_norm,beach_slope,cil,ciu,trend,n_points,n_points_nonan,r2_score,mae,mse,rmse,intercept,ERODIBILITY,geometry
id,,,,,,,,,,,,,,,,,
aus0001-0000,aus0001,104.347648,0.000000,0.000000,0.085,0.0545,0.2000,-1.456305,654.0,342.0,0.131951,28.721214,1321.059410,36.346381,179.407277,None,"LINESTRING (153.26555 -24.7007, 153.26938 -24...."
aus0001-0001,aus0001,93.495734,98.408334,0.002935,0.050,0.0387,0.0640,-0.833581,654.0,473.0,0.052783,26.029482,1092.336862,33.050520,210.629160,None,"LINESTRING (153.26525 -24.7019, 153.2692 -24.7..."
aus0001-0002,aus0001,82.069341,198.408334,0.005918,0.050,0.0428,0.0647,-0.282209,654.0,491.0,0.008220,22.896464,838.997233,28.965449,200.166307,None,"LINESTRING (153.26539 -24.70316, 153.26931 -24..."
aus0001-0003,aus0001,81.192757,298.402523,0.008900,0.055,0.0480,0.0659,0.089340,654.0,502.0,0.000987,20.770475,691.179232,26.290288,185.295473,None,"LINESTRING (153.26555 -24.70408, 153.26945 -24..."
aus0001-0004,aus0001,81.065473,398.402523,0.011882,0.075,0.0614,0.0922,0.361437,654.0,508.0,0.017461,19.528839,637.676213,25.252252,169.027861,None,"LINESTRING (153.2657 -24.70497, 153.26961 -24...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sar2538-0004,sar2538,NaN,NaN,NaN,NaN,NaN,NaN,-0.075520,1217.0,1129.0,0.021249,4.034425,37.749282,6.144044,244.758910,Low,"LINESTRING (9.33568 41.29334, 9.33294 41.29629)"
sar2539-0000,sar2539,NaN,NaN,NaN,NaN,NaN,NaN,0.159357,388.0,337.0,0.014258,6.685558,118.654858,10.892881,212.777700,Low,"LINESTRING (9.40022 41.3002, 9.40172 41.30362)"
sar2539-0001,sar2539,NaN,NaN,NaN,NaN,NaN,NaN,0.160824,388.0,356.0,0.020390,5.452777,82.388602,9.076817,191.174425,Low,"LINESTRING (9.39917 41.3019, 9.4035 41.30039)"


In [17]:
transects[transects.site_id.str.startswith("sar") & ~transects.trend.isna()]

,site_id,orientation,along_dist,along_dist_norm,beach_slope,cil,ciu,trend,n_points,n_points_nonan,r2_score,mae,mse,rmse,intercept,ERODIBILITY,geometry
id,,,,,,,,,,,,,,,,,
sar0001-0000,sar0001,NaN,NaN,NaN,NaN,NaN,NaN,0.058752,662.0,506.0,0.000083,52.753893,6159.822821,78.484539,124.312640,Medium,"LINESTRING (8.40852 38.86175, 8.40882 38.86535)"
sar0001-0001,sar0001,NaN,NaN,NaN,NaN,NaN,NaN,-0.194687,662.0,496.0,0.002706,20.017379,2018.022136,44.922401,197.142264,Medium,"LINESTRING (8.4084 38.86162, 8.41092 38.86464)"
sar0001-0002,sar0001,NaN,NaN,NaN,NaN,NaN,NaN,-0.003330,662.0,543.0,0.000008,11.757344,215.341597,14.674522,206.320761,Medium,"LINESTRING (8.40893 38.86153, 8.41236 38.86393)"
sar0001-0003,sar0001,NaN,NaN,NaN,NaN,NaN,NaN,-0.034106,662.0,595.0,0.001495,8.436936,111.945826,10.580445,239.212605,Medium,"LINESTRING (8.40904 38.86129, 8.41333 38.8626)"
sar0001-0004,sar0001,NaN,NaN,NaN,NaN,NaN,NaN,0.018929,662.0,601.0,0.000550,6.914179,95.073703,9.750574,258.243333,Medium,"LINESTRING (8.40904 38.86176, 8.4133 38.86037)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sar2538-0004,sar2538,NaN,NaN,NaN,NaN,NaN,NaN,-0.075520,1217.0,1129.0,0.021249,4.034425,37.749282,6.144044,244.758910,Low,"LINESTRING (9.33568 41.29334, 9.33294 41.29629)"
sar2539-0000,sar2539,NaN,NaN,NaN,NaN,NaN,NaN,0.159357,388.0,337.0,0.014258,6.685558,118.654858,10.892881,212.777700,Low,"LINESTRING (9.40022 41.3002, 9.40172 41.30362)"
sar2539-0001,sar2539,NaN,NaN,NaN,NaN,NaN,NaN,0.160824,388.0,356.0,0.020390,5.452777,82.388602,9.076817,191.174425,Low,"LINESTRING (9.39917 41.3019, 9.4035 41.30039)"


In [18]:
transects.drop_duplicates().to_file("transects_extended.geojson", driver="GeoJSON")